In [ ]:
COLLECTION_NAME = "Verkehr2"        
OUT_CSV_PATH = "./export/ExportVerkehr2.csv"

In [2]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd
from tqdm import tqdm

In [3]:
load_dotenv()

mongo_user = os.getenv("MONGO_USER")
mongo_pass = os.getenv("MONGO_PASS")
mongo_host = os.getenv("MONGO_HOST", "localhost")
mongo_port = os.getenv("MONGO_PORT", "27017")
mongo_db   = os.getenv("MONGO_DB")

if not all([mongo_user, mongo_pass, mongo_host, mongo_port, mongo_db]):
    raise ValueError("Mongo ENV Variablen unvollständig. Bitte .env prüfen: MONGO_USER, MONGO_PASS, MONGO_HOST, MONGO_PORT, MONGO_DB")

uri = f"mongodb://{mongo_user}:{mongo_pass}@{mongo_host}:{mongo_port}/?authSource=admin"
client = MongoClient(uri)
db = client[mongo_db]

db.command("ping")
print("MongoDB Verbindung OK:", mongo_host, mongo_port, "DB:", mongo_db)

MongoDB Verbindung OK: localhost 27017 DB: LuftqualitaetDB


In [ ]:
from typing import Optional, Dict, Any, List

def collection_to_df(
    collection_name: str,
    query: Optional[Dict[str, Any]] = None,
    projection: Optional[Dict[str, int]] = None,
    batch_size: int = 5000,
    limit: Optional[int] = None
) -> pd.DataFrame:
    col = db[collection_name]
    query = query or {}

    cursor = col.find(query, projection=projection, batch_size=batch_size)
    if limit is not None:
        cursor = cursor.limit(int(limit))

    rows: List[dict] = []
    for doc in tqdm(cursor, desc=f"Read -> {collection_name}"):
        # Mongo _id ist ein Objecz für CSV = string
        if "_id" in doc:
            doc["_id"] = str(doc["_id"])
        rows.append(doc)

    return pd.DataFrame(rows)

In [ ]:
def save_df_as_csv(
    df: pd.DataFrame,
    out_path: str,
    sep: str = ";",
    decimal: str = ",",
    encoding: str = "utf-8"
):

    df.to_csv(out_path, index=False, sep=sep, decimal=decimal, encoding=encoding)

In [ ]:
BATCH_SIZE = 5000


QUERY = {}          
PROJECTION = None    
LIMIT = None        

df = collection_to_df(
    COLLECTION_NAME,
    query=QUERY,
    projection=PROJECTION,
    batch_size=BATCH_SIZE,
    limit=LIMIT
)

print("Daten geladen. Shape:", df.shape)
display(df.head())

Read -> Verkehr2: 10it [00:00, 5352.61it/s]

Daten geladen. Shape: (10, 31)


,_id,1980,1990,2000,2001,2002,2003,2004,2005,2006,...,2018,2019,2020,2021,2022,2023,2024,Bundesland,Unnamed: 28,_imported_at
0,6960fb9f07813795b1f3129f,277000,393000,551000,565000,548000,559000,570000,575000,581000,...,668000,675000,681000,684000,679000,683000,692000,Burgenland,NaN,2026-01-09 12:59:11.483
1,6960fb9f07813795b1f312a0,296000,390000,554000,567000,516000,524000,534000,541000,547000,...,641000,648000,654000,656000,654000,656000,661000,Kaernten,NaN,2026-01-09 12:59:11.483
2,6960fb9f07813795b1f312a1,307000,431000,560000,570000,550000,561000,566000,570000,574000,...,649000,654000,659000,661000,655000,657000,662000,Niederoesterreich,NaN,2026-01-09 12:59:11.483
3,6960fb9f07813795b1f312a2,305000,416000,531000,542000,522000,531000,535000,541000,546000,...,630000,636000,641000,643000,639000,641000,645000,Oberoesterreich,NaN,2026-01-09 12:59:11.483
4,6960fb9f07813795b1f312a3,321000,418000,479000,487000,474000,483000,482000,484000,488000,...,564000,567000,570000,572000,569000,569000,573000,Salzburg,NaN,2026-01-09 12:59:11.483


In [7]:
import os
os.makedirs(os.path.dirname(OUT_CSV_PATH), exist_ok=True)

save_df_as_csv(df, OUT_CSV_PATH, sep=";", decimal=",")
print("Export gespeichert unter:", OUT_CSV_PATH)

Export gespeichert unter: ./export/ExportVerkehr2.csv
